<a href="https://colab.research.google.com/github/eagalpern/exon-foldon/blob/main/Plot_figures_1%263_all_families.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reassessing the Exon-Foldon correspondence using Frustration Analysis
Ezequiel A. Galpern, Hana Jaafari, Carlos Bueno, Peter G. Wolynes and Diego U. Ferreiro.



In [1]:
# @title Import data and install dependencies
import numpy as np
import pandas as pd
import seaborn as sns
import os
import subprocess

from matplotlib import pyplot as plt, colors
from matplotlib.colors import Normalize, BoundaryNorm
from matplotlib import cm
from matplotlib.ticker import MaxNLocator
import matplotlib.patches as mpatch
from matplotlib.collections import PatchCollection
import matplotlib.path as mpath
import matplotlib
import scipy.signal as scs

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output

# Install py3Dmol for visualizing structures
!pip install --quiet py3Dmol
import py3Dmol

# Clone the GitHub repository

REPO_URL = "https://github.com/eagalpern/exon-foldon.git"
CLONE_DIR = "exon_foldon"  # Name of the local directory where the repository will be cloned
if not os.path.exists(CLONE_DIR):
  #!git clone $REPO_URL $CLONE_DIR
  subprocess.run(f'git clone {REPO_URL} {CLONE_DIR}', shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

from exon_foldon.scripts.visualization import *


summ_all_mut=pd.read_csv('exon_foldon/data/exon_total_frustra_allfam.csv',index_col=0)
summaries=[]
for family in summ_all_mut.family.unique():
    summary = summ_all_mut[summ_all_mut.family == family].copy()  # Create a copy of the DataFrame
    summary['exon_len']=summary.exon_end_pdb-summary.exon_start_pdb

    summary['abundance_rank']=summary.abundance.rank(ascending=False)
    summary.sort_values('abundance_rank',inplace=True)
    new_cmap,colors_ = rand_cmap(len(summary), type='bright', start_with_cmap=True,
                                 first_color_black=False, last_color_black=False, verbose=False)

    if len(colors_)>len(summary):
      colors_=colors_[:len(summary)]
    summary['colors']=colors_
    summary.sort_values('abundance',inplace=True,ascending=False)
    summaries.append(summary)
tables = dict(zip(summ_all_mut.family.unique(), summaries))

position_table = pd.read_csv('exon_foldon/data/exon_freq_allfam.csv', index_col=0)
position_table['exon'] = pd.to_numeric(position_table['exon'], errors='coerce').astype('Int64')
position_table_ls=[]
for family in position_table.family.unique():
    position_table_ls.append(position_table[position_table.family == family])
position_tables = dict(zip(position_table.family.unique(), position_table_ls))

table_1=pd.read_csv('exon_foldon/data/table_1.csv')

In [3]:
# @title Figure 1. Exon characterization


family_dropdown = widgets.Dropdown(options=list(summ_all_mut.sort_values('family').family.unique()), description='Family:')
f_dropdown = widgets.Dropdown(options=['abundance', 'delta_f'], description='Plot Type:')
exon_i_dropdown= widgets.Dropdown(options=np.arange(1,100), description='Exon:')
s_exon_checkbox = widgets.Checkbox(value=False, description='Show all exons')


output = widgets.Output()

def plot_exon_view(change):
    with output:
        clear_output(wait=True)
        family = family_dropdown.value
        f_ = f_dropdown.value
        show_all_exons=s_exon_checkbox.value
        exon_i=exon_i_dropdown.value-1
        summary = tables[family]
        pdb_code=table_1.pdb[table_1.name==family].values[0]
        pdb_file='exon_foldon/data/pdb_files/'+pdb_code+'_cleaned.pdb'

        real_pdb_beg=position_tables[family].pdb_original_res_num[0]-1



        summary.index=range(len(summary))
        if exon_i<len(summary):

          fig, ax = plt.subplots(figsize=(8, 2.5))
          if f_=='abundance':
            ax.set_yscale('log')
            ax.set_ylim([0.005,1.3])
          else:
            ax.set_ylim([min(0,summary[f_][exon_i]-0.5),max(8,summary[f_][exon_i]+0.5)])
          ax.set_xlabel('Sequence position')
          ax.set_ylabel(f_)
          ax.set_title(family)

          view_positions=np.arange(summary.exon_start_pdb[exon_i],summary.exon_end_pdb[exon_i]+1)
          view_colors=np.repeat(matplotlib.colors.rgb2hex((summary['colors'][exon_i])),
                          summary.exon_len[exon_i]+1)
          # Clear the previous 3D view and create a new one
          view = view_3d_exon_hist(view_positions, view_colors, pdb_file)
          if show_all_exons:
            for i in np.arange(min(100, len(summary))):
                ax.hlines(summary[f_][i], summary.exon_start_pdb[i], summary.exon_end_pdb[i] + 1, linewidth=1.5, color=summary['colors'][i])
          else:
            ax.hlines(summary[f_][exon_i], summary.exon_start_pdb[exon_i], summary.exon_end_pdb[exon_i] + 1, linewidth=2.5, color=summary['colors'][exon_i])

          xfin=summary.exon_end_pdb.max()+2
          xini=summary.exon_start_pdb.min()
          ax.set_xlim([xini-1,xfin])
          current_xticks= [x for x in range((xini + 9) // 10 * 10, round(xfin+10,-1),20)]
          ax.set_xticks(current_xticks,current_xticks+real_pdb_beg)
          plt.show()
          view.show()

        else:
          print('Exon index out of range')



exon_i_dropdown.observe(plot_exon_view, names='value')
family_dropdown.observe(plot_exon_view, names='value')
f_dropdown.observe(plot_exon_view, names='value')
s_exon_checkbox.observe(plot_exon_view, names='value')
display(family_dropdown, f_dropdown,exon_i_dropdown, s_exon_checkbox,output)
plot_exon_view(None)  # Initial plot



Dropdown(description='Family:', options=('ACBP', 'ADH_N', 'Barstar', 'Beta-lactamase2', 'Copper-bind', 'Crysta…

Dropdown(description='Plot Type:', options=('abundance', 'delta_f'), value='abundance')

Dropdown(description='Exon:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

Checkbox(value=False, description='Show all exons')

Output()

In [6]:
# @title Figure 3. Exon boundary histogram and minimal common exons definition.

family_dropdown = widgets.Dropdown(options=list(table_1.name.values), description='Family:')
order_slider = widgets.IntSlider(
    value=10,
    min=0,
    max=30,
    description='Order:'
)
threshold_slider = widgets.FloatSlider(
    value=0.01,
    min=0,
    max=0.1,
    step=0.01,
    description='Threshold:'
)



output = widgets.Output()

def plot_exon_view(change):
    with output:
      clear_output(wait=True)

      order=order_slider.value
      border=order_slider.value
      threshold=threshold_slider.value
      family=family_dropdown.value
      size=11

      pdb_code=table_1.pdb[table_1.name==family].values[0]
      pdb_file='exon_foldon/data/pdb_files/'+pdb_code+'_cleaned.pdb'

      df=position_tables[family]


      positions=~np.isnan(df.exon_freq)
      exon_freq=df.exon_freq[positions].values
      first_res_pos=df.res_num[positions].values[0]-1
      ali_seq_num_pdb=df.pdb_original_res_num[positions].values

      final_bs=find_common_bs(exon_freq,order=order,thresh=threshold,border=order,npos=len(ali_seq_num_pdb))
      new_exons=-np.ones(len(df),dtype=int)

      if len(final_bs)>2:
        fig = plt.figure(figsize=(8,2.5))
        gs = matplotlib.gridspec.GridSpec(7, 1, wspace=0, hspace=0) #  grid
        ax0 = fig.add_subplot(gs[0, 0])
        ax1 = fig.add_subplot(gs[1:, 0])

        for i in range(len(final_bs)-1):
            new_exons[(final_bs[i]+first_res_pos):(final_bs[i+1]+first_res_pos)]=i
        df = df.assign(exon=new_exons)


        ax1.set_xlabel('Sequence position')
        ax1.set_ylabel('Density')
        ax1.set_xticks(ali_seq_num_pdb[0]+np.hstack([final_bs[:-1],np.array(final_bs[-1]-1)]))

        ax1.bar(x=ali_seq_num_pdb,height=exon_freq,color='k')

        ax1.scatter(x=ali_seq_num_pdb[0]+final_bs[1:-1],
                    y=exon_freq[final_bs[1:-1]]+max(exon_freq[final_bs[1:-1]])*0.08,
                    color='red',s=20,marker='*')


        ax1.hlines(y=threshold, xmin=ali_seq_num_pdb[0], xmax=ali_seq_num_pdb[-1], color='grey',lw=0.5,ls='--')
        ax1.vlines(x=border+ali_seq_num_pdb[0], ymin=0, ymax=max(exon_freq)*1.2, color='grey',lw=0.5,ls='--')
        ax1.vlines(x=ali_seq_num_pdb[-1]-border, ymin=0, ymax=max(exon_freq)*1.2, color='grey',lw=0.5,ls='--')
        ax1.tick_params(axis='both', which='major',labelsize=size)

        ax2=ax1.twinx()
        ax1.set_xlim([df.pdb_original_res_num.values[0],df.pdb_original_res_num.values[-1]])
        ax2.plot(df.pdb_original_res_num  ,df.fwin_average)
        ax2.fill_between(df.pdb_original_res_num ,df.fwin_average-df.fwin_std,df.fwin_average+df.fwin_std,alpha=0.2)
        ax2.set_ylabel('Frustration')
        ax2.tick_params(axis='both', which='major',labelsize=size)

        new_cmap,colors = rand_cmap(len(final_bs), type='bright', first_color_black=False, last_color_black=False, verbose=False)



        plot_ss(df,colors,ax0)
        ax0.set_axis_off()
        ax1.set_zorder(1)
        ax2.set_zorder(2)

        pdb_code=table_1.pdb[table_1.name==family].values[0]
        pdb_file='exon_foldon/data/pdb_files/'+pdb_code+'_cleaned.pdb'
        colors_=np.hstack([np.repeat(matplotlib.colors.rgb2hex(colors[i]),final_bs[i+1]-final_bs[i]) for i in range(len(final_bs)-1)])
        view=view_3d_exon_hist(ali_seq_num_pdb-df.pdb_original_res_num.values[0]+1,colors_,pdb_file)



        plt.show()
        view.show()
      else:
        print('No hotspots')


#widgets.VBox([family_dropdown, order_slider, threshold_slider])

family_dropdown.observe(plot_exon_view, names='value')
order_slider.observe(plot_exon_view, names='value')
threshold_slider.observe(plot_exon_view, names='value')
display(family_dropdown, order_slider, threshold_slider,output)
plot_exon_view(None)  # Initial plot



Dropdown(description='Family:', options=('ACBP', 'ADH_N', 'Barstar', 'Beta-lactamase2', 'Copper-bind', 'Crysta…

IntSlider(value=10, description='Order:', max=30)

FloatSlider(value=0.01, description='Threshold:', max=0.1, step=0.01)

Output()